In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import numpy as np
import pickle
import tensorflow
import json
import random
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
pickle.dump(model, open('model.pkl', 'wb'))
print('done')

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


14/14 [==============================] - 1s 5ms/step - loss: 2.6611 - accuracy: 0.0441
Epoch 2/300
14/14 [==============================] - 0s 2ms/step - loss: 2.5672 - accuracy: 0.1618
Epoch 3/300
14/14 [==============================] - 0s 2ms/step - loss: 2.5285 - accuracy: 0.1324
Epoch 4/300
14/14 [==============================] - 0s 2ms/step - loss: 2.4892 - accuracy: 0.2206
Epoch 5/300
14/14 [==============================] - 0s 2ms/step - loss: 2.3831 - accuracy: 0.2353
Epoch 6/300
14/14 [==============================] - 0s 3ms/step - loss: 2.2679 - accuracy: 0.2059
Epoch 7/300
14/14 [==============================] - 0s 4ms/step - loss: 2.2033 - accuracy: 0.3235
Epoch 8/300
14/14 [==============================] - 0s 3ms/step - loss: 2.1862 - accuracy: 0.2500
Epoch 9/300
14/14 [==============================] - 0s 3ms/step - loss: 2.0186 - accuracy: 0.3529
Epoch 10/300
14/14 [==============================] - 0s 2ms/step - loss: 1.9003 - accuracy: 0.4265
Epoch 11/300
14/14 [=

#Business problem definition:
We are going to build a web that user will input some feeling or movies name, and the chatbot will give back some movies depends on what feeling and what genres of the movies the user turn in. Also, the chatbot will answer about the overview of the movies.

#Data ingestion:
Data ingestion is already did on the above code that we create some conversation and response for the chatbot to training, and we save the tags as classes.pkl, response as intents for later the web app can load and use.

#Exploratory data analysis,

In [8]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

print("intents:", intents)
print("words:", words)
print("classes", classes)



intents: {'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'yo', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!']}, {'tag': 'age', 'patterns': ['How old', 'How old is tim', 'What is your age', 'How old are you', 'Age?'], 'responses': ['I am 21 years old!', '21 years young!']}, {'tag': 'name', 'patterns': ['What is your name', 'What should I call you', 'Whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."]}, {'tag': 'hours', 'patterns': ['When are you guys open', 'What are your hours', 'Hours of operation'], 'responses': ['We are open 9am-4pm Monday-Friday!']}, {'tag': 'movies', 'patterns': ['What type of genres do you have', 'What is popular', 'What do you recommend', 'W'], '

#Data cleanup,Model training,and Model evaluation:
We all show on the above code. we use nltk to lemmatize each words of the intents, then we put them to a set that there will be no duplicate values, then we convert each words to number for puting to neural network to training the model. Finally, we got the accuracy around 90%.